In [1]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.signal import savgol_filter
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc
from scipy.stats import linregress


def get_data_from_mat_for_calcu(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:]
    return var.T

def get_data_from_nc(file, variable):
    file_obj = nc.Dataset(file)
    data = file_obj.variables[variable]
    var_data = np.array(data)
    var_data = var_data[:, 29:, :]
    var_data[var_data > 1000000] = np.nan
    var_data = np.squeeze(var_data)
    return var_data

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:] 
    return var

ar6_region = get_data_from_mat_for_calcu('yi_yao_IRRMIP/ar6_region.mat','ar6_region')
AREA = get_data_from_mat('yi_yao_IRRMIP/input_data/AREA.mat', 'AREA') # Area for CLM_data
irr_diff = get_data_from_mat_for_calcu('yi_yao_IRRMIP/irr_diff_out.mat', 'irr_diff_out')

In [2]:
data_surface = Data_from_nc('surfdata_irrigation_method.nc')   #load the data
data_irrigation_method = data_surface.load_variable('irrigation_method')
data_lon = data_surface.load_variable('LONGXY')
data_lon = data_lon[0,:]
data_lat = data_surface.load_variable('LATIXY')
data_lat = data_lat[29:,0]
data_lat = data_lat[:]


def get_data_CESM2(variable):
    
    IWW_IRR_1901_2014_CESM2_IRR01 = get_data_from_nc('CESM2' + '_IRR01_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_NOI01 = get_data_from_nc('CESM2' + '_NOI01_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_IRR02 = get_data_from_nc('CESM2' + '_IRR02_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_NOI02 = get_data_from_nc('CESM2' + '_NOI02_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_IRR03 = get_data_from_nc('CESM2' + '_IRR03_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_NOI03 = get_data_from_nc('CESM2' + '_NOI03_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2 = (IWW_IRR_1901_2014_CESM2_IRR01 + IWW_IRR_1901_2014_CESM2_IRR02 + IWW_IRR_1901_2014_CESM2_IRR03) / 3 * 365 * 86400
    IWW_NOI_1901_2014_CESM2 = (IWW_NOI_1901_2014_CESM2_NOI01 + IWW_NOI_1901_2014_CESM2_NOI02 + IWW_NOI_1901_2014_CESM2_NOI03) / 3 * 365 * 86400
    
    return IWW_IRR_1901_2014_CESM2[:-1,:,:], IWW_NOI_1901_2014_CESM2[:-1,:,:]

def get_data_E3SM(variable3):
    
    IWW_IRR_1901_2014_E3SM_IRR01 = get_data_from_nc('E3SM_IRR01_1901_2014_'+ variable3 + '_yearmean_0.9x1.25', variable3)
    IWW_NOI_1901_2014_E3SM_NOI01 = get_data_from_nc('E3SM_NOI01_1901_2014_'+ variable3 + '_yearmean_0.9x1.25', variable3)

    IWW_IRR_1901_2014_E3SM_IRR02 = get_data_from_nc('E3SM_IRR02_1901_2014_'+ variable3 + '_yearmean_0.9x1.25', variable3)
    IWW_NOI_1901_2014_E3SM_NOI02 = get_data_from_nc('E3SM_NOI02_1901_2014_'+ variable3 + '_yearmean_0.9x1.25', variable3)

    IWW_IRR_1901_2014_E3SM = (IWW_IRR_1901_2014_E3SM_IRR01 + IWW_IRR_1901_2014_E3SM_IRR02) / 2 * 365 * 86400
    IWW_NOI_1901_2014_E3SM = (IWW_NOI_1901_2014_E3SM_NOI01 + IWW_NOI_1901_2014_E3SM_NOI02) / 2 * 365 * 86400

    return IWW_IRR_1901_2014_E3SM[6:-1,:,:], IWW_NOI_1901_2014_E3SM[6:-1,:,:]

def get_data_CESM2_gw(variable):
    
    IWW_IRR_1901_2014_CESM2_gw_IRR01 = get_data_from_nc('CESM2_gw' + '_IRR01_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_gw_NOI01 = get_data_from_nc('CESM2_gw' + '_NOI01_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_gw_IRR02 = get_data_from_nc('CESM2_gw' + '_IRR02_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_gw_NOI02 = get_data_from_nc('CESM2_gw' + '_NOI02_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_gw_IRR03 = get_data_from_nc('CESM2_gw' + '_IRR03_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_gw_NOI03 = get_data_from_nc('CESM2_gw' + '_NOI03_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_gw = (IWW_IRR_1901_2014_CESM2_gw_IRR01 + IWW_IRR_1901_2014_CESM2_gw_IRR02 + IWW_IRR_1901_2014_CESM2_gw_IRR03) / 3 * 365 * 86400
    IWW_NOI_1901_2014_CESM2_gw = (IWW_NOI_1901_2014_CESM2_gw_NOI01 + IWW_NOI_1901_2014_CESM2_gw_NOI02 + IWW_NOI_1901_2014_CESM2_gw_NOI03) / 3 * 365 * 86400
    
    return IWW_IRR_1901_2014_CESM2_gw[:-1,:,:], IWW_NOI_1901_2014_CESM2_gw[:-1,:,:]
    
def get_data_NorESM(variable):
    IWW_IRR_1901_2014_NorESM_IRR01 = get_data_from_nc('NorESM' + '_IRR01_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_NorESM_NOI01 = get_data_from_nc('NorESM' + '_NOI01_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_NorESM_IRR02 = get_data_from_nc('NorESM' + '_IRR02_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_NorESM_NOI02 = get_data_from_nc('NorESM' + '_NOI02_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_NorESM_IRR03 = get_data_from_nc('NorESM' + '_IRR03_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_NorESM_NOI03 = get_data_from_nc('NorESM' + '_NOI03_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_NorESM_IRR01_temp = np.zeros([116,163,288])
    IWW_IRR_1901_2014_NorESM_IRR01_temp[1:, :, :] = IWW_IRR_1901_2014_NorESM_IRR01
    IWW_IRR_1901_2014_NorESM_IRR01_temp[0, :, :] = (IWW_IRR_1901_2014_NorESM_IRR02[0, :, :] + IWW_IRR_1901_2014_NorESM_IRR03[0, :, :])/2
    IWW_IRR_1901_2014_NorESM_IRR01 = IWW_IRR_1901_2014_NorESM_IRR01_temp
    
    IWW_IRR_1901_2014_NorESM = (IWW_IRR_1901_2014_NorESM_IRR01 + IWW_IRR_1901_2014_NorESM_IRR02 + IWW_IRR_1901_2014_NorESM_IRR03) / 3 * 365 * 86400
    IWW_NOI_1901_2014_NorESM = (IWW_NOI_1901_2014_NorESM_NOI01 + IWW_NOI_1901_2014_NorESM_NOI02 + IWW_NOI_1901_2014_NorESM_NOI03) / 3 * 365 * 86400
    
    return IWW_IRR_1901_2014_NorESM[1:-1,:,:], IWW_NOI_1901_2014_NorESM[1:-1,:,:]
    
    
    
def get_data_IPSL(variable2):
    
    IWW_IRR_1901_2014_IPSL = get_data_from_nc('IRR01_'+ variable2 + '_1901_2014_Month.nc_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_IPSL = get_data_from_nc('NOI01_'+ variable2 + '_1901_2014_Month.nc_yearmean_0.9x1.25', variable2)


    IWW_IRR_1901_2014_IPSL = (IWW_IRR_1901_2014_IPSL) * 365 * 86400
    IWW_NOI_1901_2014_IPSL = (IWW_NOI_1901_2014_IPSL) * 365 * 86400

    return IWW_IRR_1901_2014_IPSL, IWW_NOI_1901_2014_IPSL

def get_data_CNRM(variable2):
    
    IWW_IRR_1901_2014_CNRM_IRR01 = get_data_from_nc(variable2 + '_IRR01.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI01 = get_data_from_nc(variable2 + '_NOI01.nc_yearmean_yearmean_0.9x1.25', variable2)

    IWW_IRR_1901_2014_CNRM_IRR02 = get_data_from_nc(variable2 + '_IRR02.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI02 = get_data_from_nc(variable2 + '_NOI02.nc_yearmean_yearmean_0.9x1.25', variable2)

    IWW_IRR_1901_2014_CNRM_IRR03 = get_data_from_nc(variable2 + '_IRR03.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI03 = get_data_from_nc(variable2 + '_NOI03.nc_yearmean_yearmean_0.9x1.25', variable2)

    IWW_IRR_1901_2014_CNRM_IRR04 = get_data_from_nc(variable2 + '_IRR04.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI04 = get_data_from_nc(variable2 + '_NOI04.nc_yearmean_yearmean_0.9x1.25', variable2)

    IWW_IRR_1901_2014_CNRM_IRR05 = get_data_from_nc(variable2 + '_IRR05.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI05 = get_data_from_nc(variable2 + '_NOI05.nc_yearmean_yearmean_0.9x1.25', variable2)

 
    
    IWW_IRR_1901_2014_CNRM = (IWW_IRR_1901_2014_CNRM_IRR01+IWW_IRR_1901_2014_CNRM_IRR02+IWW_IRR_1901_2014_CNRM_IRR03+IWW_IRR_1901_2014_CNRM_IRR04+IWW_IRR_1901_2014_CNRM_IRR05) / 5 * 365 * 86400
    IWW_NOI_1901_2014_CNRM = (IWW_NOI_1901_2014_CNRM_NOI01+IWW_NOI_1901_2014_CNRM_NOI02+IWW_NOI_1901_2014_CNRM_NOI03+IWW_NOI_1901_2014_CNRM_NOI04+IWW_NOI_1901_2014_CNRM_NOI05) / 5 * 365 * 86400

#     IWW_IRR_1901_2014_CNRM = (IWW_IRR_1901_2014_CNRM_IRR02+IWW_IRR_1901_2014_CNRM_IRR03+IWW_IRR_1901_2014_CNRM_IRR04+IWW_IRR_1901_2014_CNRM_IRR05) / 4 * 365 * 86400
#     IWW_NOI_1901_2014_CNRM = (IWW_NOI_1901_2014_CNRM_NOI02+IWW_NOI_1901_2014_CNRM_NOI03+IWW_NOI_1901_2014_CNRM_NOI04+IWW_NOI_1901_2014_CNRM_NOI05) / 4 * 365 * 86400

    return IWW_IRR_1901_2014_CNRM, IWW_NOI_1901_2014_CNRM
    
    
def get_data_MIROC(variable):
    
    IWW_IRR_1901_2014_MIROC_IRR01 = get_data_from_nc(variable + '_mon_MIROC_IRR01_1901-2014.nc_0.9x1.25_yearmean', variable)
    IWW_NOI_1901_2014_MIROC_NOI01 = get_data_from_nc(variable + '_mon_MIROC_NOI01_1901-2014.nc_0.9x1.25_yearmean', variable)

    IWW_IRR_1901_2014_MIROC_IRR02 = get_data_from_nc(variable + '_mon_MIROC_IRR02_1901-2014.nc_0.9x1.25_yearmean', variable)
    IWW_NOI_1901_2014_MIROC_NOI02 = get_data_from_nc(variable + '_mon_MIROC_NOI02_1901-2014.nc_0.9x1.25_yearmean', variable)

    IWW_IRR_1901_2014_MIROC_IRR03 = get_data_from_nc(variable + '_mon_MIROC_IRR03_1901-2014.nc_0.9x1.25_yearmean', variable)
    IWW_NOI_1901_2014_MIROC_NOI03 = get_data_from_nc(variable + '_mon_MIROC_NOI03_1901-2014.nc_0.9x1.25_yearmean', variable)

    IWW_IRR_1901_2014_MIROC = (IWW_IRR_1901_2014_MIROC_IRR01 + IWW_IRR_1901_2014_MIROC_IRR02 + IWW_IRR_1901_2014_MIROC_IRR03) / 3 * 365 * 86400
    IWW_NOI_1901_2014_MIROC = (IWW_NOI_1901_2014_MIROC_NOI01 + IWW_NOI_1901_2014_MIROC_NOI02 + IWW_NOI_1901_2014_MIROC_NOI03) / 3 * 365 * 86400
    
    return IWW_IRR_1901_2014_MIROC, IWW_NOI_1901_2014_MIROC

#     All_forcings_all = (All_forcings_CESM2 + All_forcings_CESM2_gw + All_forcings_NorESM + All_forcings_CNRM + All_forcings_IPSL) / 6
#     Oth_forcings_all = (Oth_forcings_CESM2 + Oth_forcings_CESM2_gw + Oth_forcings_NorESM + Oth_forcings_CNRM + Oth_forcings_IPSL) / 6
#     Irr_forcings_all = (Irr_forcings_CESM2 + Irr_forcings_CESM2_gw + Irr_forcings_NorESM + Irr_forcings_CNRM + Irr_forcings_IPSL) / 6

def calcu_global_Water(IWW_1901_2014, AREA):
    area_for_calcu = AREA.T
    area_for_calcu[np.isnan(IWW_1901_2014[0, :, :])] = np.nan
    IWW_globe_1901_2014 = IWW_1901_2014 * AREA.T
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    area_for_calcu = np.nansum(area_for_calcu, axis=1)
    area_for_calcu = np.nansum(area_for_calcu, axis=0)
    
    return IWW_globe_1901_2014/area_for_calcu

def calcu_Water_region(IWW_1901_2014, AREA, region_id):
    AREA = get_data_from_mat('AREA.mat', 'AREA') # Area for CLM_data
    area_for_calcu = AREA.T
    
    area_for_calcu[np.isnan(IWW_1901_2014[0, :, :])] = np.nan
    area_for_calcu[ar6_region <= region_id-0.2] = np.nan
    area_for_calcu[ar6_region >= region_id+0.2] = np.nan
    
    IWW_globe_1901_2014 = IWW_1901_2014 * area_for_calcu
    IWW_globe_1901_2014[:, ar6_region<=region_id-0.2] = np.nan
    IWW_globe_1901_2014[:, ar6_region>=region_id+0.2] = np.nan
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    
    area_for_calcu = np.nansum(area_for_calcu, axis=1)
    area_for_calcu = np.nansum(area_for_calcu, axis=0)
    
    return IWW_globe_1901_2014/area_for_calcu

def plot_line(ax, index, data_mean, data_min, data_max, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 2, marker = marker, markersize=1.2, alpha=0.8)
    ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 16)
    plt.title(title1, loc='left', fontsize = 16)
    plt.ylabel(ylabel, fontsize = 18)
    plt.xlabel(xlabel, fontsize = 18)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2015)
    plt.legend(loc = 'upper left', fontsize = 16)

In [3]:
IWW_IRR_1901_2014_CESM2_RAIN, IWW_NOI_1901_2014_CESM2_RAIN = get_data_CESM2('RAIN_FROM_ATM')
IWW_IRR_1901_2014_CESM2_gw_RAIN, IWW_NOI_1901_2014_CESM2_gw_RAIN = get_data_CESM2_gw('RAIN_FROM_ATM')
IWW_IRR_1901_2014_NorESM_RAIN, IWW_NOI_1901_2014_NorESM_RAIN = get_data_NorESM('RAIN_FROM_ATM')
IWW_IRR_1901_2014_E3SM_RAIN, IWW_NOI_1901_2014_E3SM_RAIN = get_data_E3SM('RAIN')


IWW_IRR_1901_2014_CESM2_SNOW, IWW_NOI_1901_2014_CESM2_SNOW = get_data_CESM2('SNOW_FROM_ATM')
IWW_IRR_1901_2014_CESM2_gw_SNOW, IWW_NOI_1901_2014_CESM2_gw_SNOW = get_data_CESM2_gw('SNOW_FROM_ATM')
IWW_IRR_1901_2014_NorESM_SNOW, IWW_NOI_1901_2014_NorESM_SNOW = get_data_NorESM('SNOW_FROM_ATM')
IWW_IRR_1901_2014_E3SM_SNOW, IWW_NOI_1901_2014_E3SM_SNOW = get_data_E3SM('SNOW')


IWW_IRR_1901_2014_CESM2_P = IWW_IRR_1901_2014_CESM2_RAIN + IWW_IRR_1901_2014_CESM2_SNOW
IWW_IRR_1901_2014_CESM2_gw_P = IWW_IRR_1901_2014_CESM2_gw_RAIN + IWW_IRR_1901_2014_CESM2_gw_SNOW
IWW_IRR_1901_2014_NorESM_P = IWW_IRR_1901_2014_NorESM_RAIN + IWW_IRR_1901_2014_NorESM_SNOW
IWW_IRR_1901_2014_E3SM_P = IWW_IRR_1901_2014_E3SM_RAIN + IWW_IRR_1901_2014_E3SM_SNOW

IWW_NOI_1901_2014_CESM2_P = IWW_NOI_1901_2014_CESM2_RAIN + IWW_NOI_1901_2014_CESM2_SNOW
IWW_NOI_1901_2014_CESM2_gw_P = IWW_NOI_1901_2014_CESM2_gw_RAIN + IWW_NOI_1901_2014_CESM2_gw_SNOW
IWW_NOI_1901_2014_NorESM_P = IWW_NOI_1901_2014_NorESM_RAIN + IWW_NOI_1901_2014_NorESM_SNOW
IWW_NOI_1901_2014_E3SM_P = IWW_NOI_1901_2014_E3SM_RAIN + IWW_NOI_1901_2014_E3SM_SNOW

IWW_IRR_1901_2014_IPSL_P, IWW_NOI_1901_2014_IPSL_P = get_data_IPSL('pr')
IWW_IRR_1901_2014_CNRM_P, IWW_NOI_1901_2014_CNRM_P = get_data_CNRM('pr')
IWW_IRR_1901_2014_MIROC_P, IWW_NOI_1901_2014_MIROC_P = get_data_MIROC('pr')


IWW_IRR_1901_2014_CESM2_R, IWW_NOI_1901_2014_CESM2_R = get_data_CESM2('QRUNOFF')
IWW_IRR_1901_2014_CESM2_gw_R, IWW_NOI_1901_2014_CESM2_gw_R = get_data_CESM2_gw('QRUNOFF')
IWW_IRR_1901_2014_NorESM_R, IWW_NOI_1901_2014_NorESM_R = get_data_NorESM('QRUNOFF')
IWW_IRR_1901_2014_E3SM_R, IWW_NOI_1901_2014_E3SM_R = get_data_E3SM('QRUNOFF')

IWW_IRR_1901_2014_IPSL_R, IWW_NOI_1901_2014_IPSL_R = get_data_IPSL('mrro')
IWW_IRR_1901_2014_CNRM_R, IWW_NOI_1901_2014_CNRM_R = get_data_CNRM('mrro')
IWW_IRR_1901_2014_MIROC_R, IWW_NOI_1901_2014_MIROC_R = get_data_MIROC('mrro')

IWW_IRR_1901_2014_CESM2_ET, IWW_NOI_1901_2014_CESM2_ET = get_data_CESM2('QFLX_EVAP_TOT')
IWW_IRR_1901_2014_CESM2_gw_ET, IWW_NOI_1901_2014_CESM2_gw_ET = get_data_CESM2_gw('QFLX_EVAP_TOT')
IWW_IRR_1901_2014_NorESM_ET, IWW_NOI_1901_2014_NorESM_ET = get_data_NorESM('QFLX_EVAP_TOT')

IWW_IRR_1901_2014_E3SM_E1, IWW_NOI_1901_2014_E3SM_E1 = get_data_E3SM('QSOIL')
IWW_IRR_1901_2014_E3SM_E2, IWW_NOI_1901_2014_E3SM_E2 = get_data_E3SM('QVEGE')
IWW_IRR_1901_2014_E3SM_T, IWW_NOI_1901_2014_E3SM_T = get_data_E3SM('QVEGT')

IWW_IRR_1901_2014_E3SM_ET = IWW_IRR_1901_2014_E3SM_E1 + IWW_IRR_1901_2014_E3SM_E2 + IWW_IRR_1901_2014_E3SM_T
IWW_NOI_1901_2014_E3SM_ET = IWW_NOI_1901_2014_E3SM_E1 + IWW_NOI_1901_2014_E3SM_E2 + IWW_NOI_1901_2014_E3SM_T

IWW_IRR_1901_2014_IPSL_E, IWW_NOI_1901_2014_IPSL_E = get_data_IPSL('evspsbl')
IWW_IRR_1901_2014_CNRM_E, IWW_NOI_1901_2014_CNRM_E = get_data_CNRM('evspsbl')
IWW_IRR_1901_2014_MIROC_E, IWW_NOI_1901_2014_MIROC_E = get_data_MIROC('evspsbl')

IWW_IRR_1901_2014_IPSL_T, IWW_NOI_1901_2014_IPSL_T = get_data_IPSL('tran')
IWW_IRR_1901_2014_CNRM_T, IWW_NOI_1901_2014_CNRM_T = get_data_CNRM('tran')
IWW_IRR_1901_2014_MIROC_T, IWW_NOI_1901_2014_MIROC_T = get_data_MIROC('tran')

IWW_IRR_1901_2014_IPSL_ET = IWW_IRR_1901_2014_IPSL_E
IWW_IRR_1901_2014_CNRM_ET = IWW_IRR_1901_2014_CNRM_E
IWW_IRR_1901_2014_MIROC_ET = IWW_IRR_1901_2014_MIROC_E + IWW_IRR_1901_2014_MIROC_T
IWW_NOI_1901_2014_IPSL_ET = IWW_NOI_1901_2014_IPSL_E
IWW_NOI_1901_2014_CNRM_ET = IWW_NOI_1901_2014_CNRM_E
IWW_NOI_1901_2014_MIROC_ET = IWW_NOI_1901_2014_MIROC_E + IWW_NOI_1901_2014_MIROC_T

In [4]:
IWW_IRR_1901_2014_E3SM_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_IRR_1901_2014_IPSL_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_IRR_1901_2014_CNRM_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_IRR_1901_2014_MIROC_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan

IWW_NOI_1901_2014_E3SM_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_NOI_1901_2014_IPSL_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_NOI_1901_2014_CNRM_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_NOI_1901_2014_MIROC_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan

IWW_IRR_1901_2014_E3SM_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_IPSL_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_CNRM_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_MIROC_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan

IWW_NOI_1901_2014_E3SM_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_NOI_1901_2014_IPSL_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_NOI_1901_2014_CNRM_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_NOI_1901_2014_MIROC_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan

IWW_IRR_1901_2014_E3SM_R[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_IPSL_R[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_CNRM_R[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_MIROC_R[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan

IWW_NOI_1901_2014_E3SM_R[np.isnan(IWW_IRR_1901_2014_CESM2_R)] = np.nan
IWW_NOI_1901_2014_IPSL_R[np.isnan(IWW_IRR_1901_2014_CESM2_R)] = np.nan
IWW_NOI_1901_2014_CNRM_R[np.isnan(IWW_IRR_1901_2014_CESM2_R)] = np.nan
IWW_NOI_1901_2014_MIROC_R[np.isnan(IWW_IRR_1901_2014_CESM2_R)] = np.nan

In [5]:
def get_regional_P(region_id):
    IWW_IRR_1901_2014_CESM2_P_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_P, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_P_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_P, AREA, region_id)
#     IWW_IRR_1901_2014_CESM2_P_global = IWW_IRR_1901_2014_CESM2_P_global - np.mean(IWW_NOI_1901_2014_CESM2_P_global[0:114])
#     IWW_NOI_1901_2014_CESM2_P_global = IWW_NOI_1901_2014_CESM2_P_global - np.mean(IWW_NOI_1901_2014_CESM2_P_global[0:114])


    IWW_IRR_1901_2014_CESM2_gw_P_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_gw_P, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_gw_P_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_gw_P, AREA, region_id)
#     IWW_IRR_1901_2014_CESM2_gw_P_global = IWW_IRR_1901_2014_CESM2_gw_P_global - np.mean(IWW_NOI_1901_2014_CESM2_gw_P_global[0:114])
#     IWW_NOI_1901_2014_CESM2_gw_P_global = IWW_NOI_1901_2014_CESM2_gw_P_global - np.mean(IWW_NOI_1901_2014_CESM2_gw_P_global[0:114])


    IWW_IRR_1901_2014_NorESM_P_global = calcu_Water_region(IWW_IRR_1901_2014_NorESM_P, AREA, region_id)
    IWW_NOI_1901_2014_NorESM_P_global = calcu_Water_region(IWW_NOI_1901_2014_NorESM_P, AREA, region_id)
#     IWW_IRR_1901_2014_NorESM_P_global = IWW_IRR_1901_2014_NorESM_P_global - np.mean(IWW_NOI_1901_2014_NorESM_P_global[0:114])
#     IWW_NOI_1901_2014_NorESM_P_global = IWW_NOI_1901_2014_NorESM_P_global - np.mean(IWW_NOI_1901_2014_NorESM_P_global[0:114])


    IWW_IRR_1901_2014_E3SM_P_global = calcu_Water_region(IWW_IRR_1901_2014_E3SM_P, AREA, region_id)
    IWW_NOI_1901_2014_E3SM_P_global = calcu_Water_region(IWW_NOI_1901_2014_E3SM_P, AREA, region_id)
#     IWW_IRR_1901_2014_E3SM_P_global = IWW_IRR_1901_2014_E3SM_P_global - np.mean(IWW_NOI_1901_2014_E3SM_P_global[0:114])
#     IWW_NOI_1901_2014_E3SM_P_global = IWW_NOI_1901_2014_E3SM_P_global - np.mean(IWW_NOI_1901_2014_E3SM_P_global[0:114])


    IWW_IRR_1901_2014_IPSL_P_global = calcu_Water_region(IWW_IRR_1901_2014_IPSL_P, AREA, region_id)
    IWW_NOI_1901_2014_IPSL_P_global = calcu_Water_region(IWW_NOI_1901_2014_IPSL_P, AREA, region_id)
#     IWW_IRR_1901_2014_IPSL_P_global = IWW_IRR_1901_2014_IPSL_P_global - np.mean(IWW_NOI_1901_2014_IPSL_P_global[0:114])
#     IWW_NOI_1901_2014_IPSL_P_global = IWW_NOI_1901_2014_IPSL_P_global - np.mean(IWW_NOI_1901_2014_IPSL_P_global[0:114])


    IWW_IRR_1901_2014_CNRM_P_global = calcu_Water_region(IWW_IRR_1901_2014_CNRM_P, AREA, region_id)
    IWW_NOI_1901_2014_CNRM_P_global = calcu_Water_region(IWW_NOI_1901_2014_CNRM_P, AREA, region_id)
#     IWW_IRR_1901_2014_CNRM_P_global = IWW_IRR_1901_2014_CNRM_P_global - np.mean(IWW_NOI_1901_2014_CNRM_P_global[0:114])
#     IWW_NOI_1901_2014_CNRM_P_global = IWW_NOI_1901_2014_CNRM_P_global - np.mean(IWW_NOI_1901_2014_CNRM_P_global[0:114])

    
    IWW_IRR_1901_2014_MIROC_P_global = calcu_Water_region(IWW_IRR_1901_2014_MIROC_P, AREA, region_id)
    IWW_NOI_1901_2014_MIROC_P_global = calcu_Water_region(IWW_NOI_1901_2014_MIROC_P, AREA, region_id)
#     IWW_IRR_1901_2014_MIROC_P_global = IWW_IRR_1901_2014_MIROC_P_global - np.mean(IWW_NOI_1901_2014_MIROC_P_global[0:114])
#     IWW_NOI_1901_2014_MIROC_P_global = IWW_NOI_1901_2014_MIROC_P_global - np.mean(IWW_NOI_1901_2014_MIROC_P_global[0:114])

    
    IWW_IRR_1901_2014_all_P = np.vstack((IWW_IRR_1901_2014_CESM2_P_global,
                               IWW_IRR_1901_2014_CESM2_gw_P_global,
                               IWW_IRR_1901_2014_NorESM_P_global,
                               IWW_IRR_1901_2014_E3SM_P_global,
                               IWW_IRR_1901_2014_IPSL_P_global,
                                         IWW_IRR_1901_2014_MIROC_P_global,
                               IWW_IRR_1901_2014_CNRM_P_global))

    IWW_NOI_1901_2014_all_P = np.vstack((IWW_NOI_1901_2014_CESM2_P_global,
                                   IWW_NOI_1901_2014_CESM2_gw_P_global,
                                   IWW_NOI_1901_2014_NorESM_P_global,
                                   IWW_NOI_1901_2014_E3SM_P_global,
                                   IWW_NOI_1901_2014_IPSL_P_global,
                                         IWW_NOI_1901_2014_MIROC_P_global,
                                   IWW_NOI_1901_2014_CNRM_P_global))
    
    IWW_IRR_NOI_1901_2014_all_P = IWW_IRR_1901_2014_all_P - IWW_NOI_1901_2014_all_P
    
    return IWW_IRR_1901_2014_all_P, IWW_NOI_1901_2014_all_P, IWW_IRR_NOI_1901_2014_all_P

IWW_IRR_1901_2014_all_P_SAS, IWW_NOI_1901_2014_all_P_SAS, IWW_IRR_NOI_1901_2014_all_P_SAS = get_regional_P(38)
IWW_IRR_1901_2014_all_P_SAS_75 = np.percentile(IWW_IRR_1901_2014_all_P_SAS, 75, axis = 0)
IWW_IRR_1901_2014_all_P_SAS_50 = np.percentile(IWW_IRR_1901_2014_all_P_SAS, 50, axis = 0)
IWW_IRR_1901_2014_all_P_SAS_mean = np.mean(IWW_IRR_1901_2014_all_P_SAS, axis = 0)
IWW_IRR_1901_2014_all_P_SAS_25 = np.percentile(IWW_IRR_1901_2014_all_P_SAS, 25, axis = 0)

IWW_NOI_1901_2014_all_P_SAS_75 = np.percentile(IWW_NOI_1901_2014_all_P_SAS, 75, axis = 0)
IWW_NOI_1901_2014_all_P_SAS_50 = np.percentile(IWW_NOI_1901_2014_all_P_SAS, 50, axis = 0)
IWW_NOI_1901_2014_all_P_SAS_mean = np.mean(IWW_NOI_1901_2014_all_P_SAS, axis = 0)
IWW_NOI_1901_2014_all_P_SAS_25 = np.percentile(IWW_NOI_1901_2014_all_P_SAS, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_SAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_SAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_SAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_SAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_SAS_mean = np.mean(IWW_IRR_NOI_1901_2014_all_P_SAS, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_SAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_SAS, 25, axis = 0)



IWW_IRR_1901_2014_all_P_EAS, IWW_NOI_1901_2014_all_P_EAS, IWW_IRR_NOI_1901_2014_all_P_EAS = get_regional_P(36)
IWW_IRR_1901_2014_all_P_EAS_75 = np.percentile(IWW_IRR_1901_2014_all_P_EAS, 75, axis = 0)
IWW_IRR_1901_2014_all_P_EAS_50 = np.percentile(IWW_IRR_1901_2014_all_P_EAS, 50, axis = 0)
IWW_IRR_1901_2014_all_P_EAS_25 = np.percentile(IWW_IRR_1901_2014_all_P_EAS, 25, axis = 0)

IWW_NOI_1901_2014_all_P_EAS_75 = np.percentile(IWW_NOI_1901_2014_all_P_EAS, 75, axis = 0)
IWW_NOI_1901_2014_all_P_EAS_50 = np.percentile(IWW_NOI_1901_2014_all_P_EAS, 50, axis = 0)
IWW_NOI_1901_2014_all_P_EAS_25 = np.percentile(IWW_NOI_1901_2014_all_P_EAS, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_EAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_EAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_EAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_EAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_EAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_EAS, 25, axis = 0)



IWW_IRR_1901_2014_all_P_MED, IWW_NOI_1901_2014_all_P_MED, IWW_IRR_NOI_1901_2014_all_P_MED = get_regional_P(20)
IWW_IRR_1901_2014_all_P_MED_75 = np.percentile(IWW_IRR_1901_2014_all_P_MED, 75, axis = 0)
IWW_IRR_1901_2014_all_P_MED_50 = np.percentile(IWW_IRR_1901_2014_all_P_MED, 50, axis = 0)
IWW_IRR_1901_2014_all_P_MED_mean = np.mean(IWW_IRR_1901_2014_all_P_MED, axis = 0)
IWW_IRR_1901_2014_all_P_MED_25 = np.percentile(IWW_IRR_1901_2014_all_P_MED, 25, axis = 0)

IWW_NOI_1901_2014_all_P_MED_75 = np.percentile(IWW_NOI_1901_2014_all_P_MED, 75, axis = 0)
IWW_NOI_1901_2014_all_P_MED_50 = np.percentile(IWW_NOI_1901_2014_all_P_MED, 50, axis = 0)
IWW_NOI_1901_2014_all_P_MED_mean = np.mean(IWW_NOI_1901_2014_all_P_MED, axis = 0)
IWW_NOI_1901_2014_all_P_MED_25 = np.percentile(IWW_NOI_1901_2014_all_P_MED, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_MED_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_MED, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_MED_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_MED, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_MED_mean = np.mean(IWW_IRR_NOI_1901_2014_all_P_MED, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_MED_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_MED, 25, axis = 0)





IWW_IRR_1901_2014_all_P_WNA, IWW_NOI_1901_2014_all_P_WNA, IWW_IRR_NOI_1901_2014_all_P_WNA = get_regional_P(4)
IWW_IRR_1901_2014_all_P_WNA_75 = np.percentile(IWW_IRR_1901_2014_all_P_WNA, 75, axis = 0)
IWW_IRR_1901_2014_all_P_WNA_50 = np.percentile(IWW_IRR_1901_2014_all_P_WNA, 50, axis = 0)
IWW_IRR_1901_2014_all_P_WNA_25 = np.percentile(IWW_IRR_1901_2014_all_P_WNA, 25, axis = 0)

IWW_NOI_1901_2014_all_P_WNA_75 = np.percentile(IWW_NOI_1901_2014_all_P_WNA, 75, axis = 0)
IWW_NOI_1901_2014_all_P_WNA_50 = np.percentile(IWW_NOI_1901_2014_all_P_WNA, 50, axis = 0)
IWW_NOI_1901_2014_all_P_WNA_25 = np.percentile(IWW_NOI_1901_2014_all_P_WNA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_WNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_WNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_WNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_WNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_WNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_WNA, 25, axis = 0)




IWW_IRR_1901_2014_all_P_CNA, IWW_NOI_1901_2014_all_P_CNA, IWW_IRR_NOI_1901_2014_all_P_CNA = get_regional_P(5)
IWW_IRR_1901_2014_all_P_CNA_75 = np.percentile(IWW_IRR_1901_2014_all_P_CNA, 75, axis = 0)
IWW_IRR_1901_2014_all_P_CNA_50 = np.percentile(IWW_IRR_1901_2014_all_P_CNA, 50, axis = 0)
IWW_IRR_1901_2014_all_P_CNA_mean = np.mean(IWW_IRR_1901_2014_all_P_CNA, axis = 0)
IWW_IRR_1901_2014_all_P_CNA_25 = np.percentile(IWW_IRR_1901_2014_all_P_CNA, 25, axis = 0)

IWW_NOI_1901_2014_all_P_CNA_75 = np.percentile(IWW_NOI_1901_2014_all_P_CNA, 75, axis = 0)
IWW_NOI_1901_2014_all_P_CNA_50 = np.percentile(IWW_NOI_1901_2014_all_P_CNA, 50, axis = 0)
IWW_NOI_1901_2014_all_P_CNA_mean = np.mean(IWW_NOI_1901_2014_all_P_CNA, axis = 0)
IWW_NOI_1901_2014_all_P_CNA_25 = np.percentile(IWW_NOI_1901_2014_all_P_CNA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_CNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_CNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_CNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_CNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_CNA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_P_CNA, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_CNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_CNA, 25, axis = 0)




IWW_IRR_1901_2014_all_P_NCA, IWW_NOI_1901_2014_all_P_NCA, IWW_IRR_NOI_1901_2014_all_P_NCA = get_regional_P(33)
IWW_IRR_1901_2014_all_P_NCA_75 = np.percentile(IWW_IRR_1901_2014_all_P_NCA, 75, axis = 0)
IWW_IRR_1901_2014_all_P_NCA_50 = np.percentile(IWW_IRR_1901_2014_all_P_NCA, 50, axis = 0)
IWW_IRR_1901_2014_all_P_NCA_mean = np.mean(IWW_IRR_1901_2014_all_P_NCA, axis = 0)
IWW_IRR_1901_2014_all_P_NCA_25 = np.percentile(IWW_IRR_1901_2014_all_P_NCA, 25, axis = 0)

IWW_NOI_1901_2014_all_P_NCA_75 = np.percentile(IWW_NOI_1901_2014_all_P_NCA, 75, axis = 0)
IWW_NOI_1901_2014_all_P_NCA_50 = np.percentile(IWW_NOI_1901_2014_all_P_NCA, 50, axis = 0)
IWW_NOI_1901_2014_all_P_NCA_mean = np.mean(IWW_NOI_1901_2014_all_P_NCA, axis = 0)
IWW_NOI_1901_2014_all_P_NCA_25 = np.percentile(IWW_NOI_1901_2014_all_P_NCA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_NCA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_NCA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_NCA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_NCA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_NCA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_P_NCA, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_NCA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_NCA, 25, axis = 0)




IWW_IRR_1901_2014_all_P_SEA, IWW_NOI_1901_2014_all_P_SEA, IWW_IRR_NOI_1901_2014_all_P_SEA = get_regional_P(39)
IWW_IRR_1901_2014_all_P_SEA_75 = np.percentile(IWW_IRR_1901_2014_all_P_SEA, 75, axis = 0)
IWW_IRR_1901_2014_all_P_SEA_50 = np.percentile(IWW_IRR_1901_2014_all_P_SEA, 50, axis = 0)

IWW_IRR_1901_2014_all_P_SEA_25 = np.percentile(IWW_IRR_1901_2014_all_P_SEA, 25, axis = 0)

IWW_NOI_1901_2014_all_P_SEA_75 = np.percentile(IWW_NOI_1901_2014_all_P_SEA, 75, axis = 0)
IWW_NOI_1901_2014_all_P_SEA_50 = np.percentile(IWW_NOI_1901_2014_all_P_SEA, 50, axis = 0)
IWW_NOI_1901_2014_all_P_SEA_25 = np.percentile(IWW_NOI_1901_2014_all_P_SEA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_SEA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_SEA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_SEA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_SEA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_SEA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_SEA, 25, axis = 0)


In [6]:
def get_regional_ET(region_id):
    IWW_IRR_1901_2014_CESM2_ET_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_ET, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_ET_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_ET, AREA, region_id)
#     IWW_IRR_1901_2014_CESM2_ET_global = IWW_IRR_1901_2014_CESM2_ET_global - np.mean(IWW_NOI_1901_2014_CESM2_ET_global[0:114])
#     IWW_NOI_1901_2014_CESM2_ET_global = IWW_NOI_1901_2014_CESM2_ET_global - np.mean(IWW_NOI_1901_2014_CESM2_ET_global[0:114])

    IWW_IRR_1901_2014_CESM2_gw_ET_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_gw_ET, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_gw_ET_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_gw_ET, AREA, region_id)

    IWW_IRR_1901_2014_NorESM_ET_global = calcu_Water_region(IWW_IRR_1901_2014_NorESM_ET, AREA, region_id)
    IWW_NOI_1901_2014_NorESM_ET_global = calcu_Water_region(IWW_NOI_1901_2014_NorESM_ET, AREA, region_id)

    IWW_IRR_1901_2014_E3SM_ET_global = calcu_Water_region(IWW_IRR_1901_2014_E3SM_ET, AREA, region_id)
    IWW_NOI_1901_2014_E3SM_ET_global = calcu_Water_region(IWW_NOI_1901_2014_E3SM_ET, AREA, region_id)

    IWW_IRR_1901_2014_IPSL_ET_global = calcu_Water_region(IWW_IRR_1901_2014_IPSL_ET, AREA, region_id)
    IWW_NOI_1901_2014_IPSL_ET_global = calcu_Water_region(IWW_NOI_1901_2014_IPSL_ET, AREA, region_id)

    IWW_IRR_1901_2014_CNRM_ET_global = calcu_Water_region(IWW_IRR_1901_2014_CNRM_ET, AREA, region_id)
    IWW_NOI_1901_2014_CNRM_ET_global = calcu_Water_region(IWW_NOI_1901_2014_CNRM_ET, AREA, region_id)
    
    IWW_IRR_1901_2014_MIROC_ET_global = calcu_Water_region(IWW_IRR_1901_2014_MIROC_ET, AREA, region_id)
    IWW_NOI_1901_2014_MIROC_ET_global = calcu_Water_region(IWW_NOI_1901_2014_MIROC_ET, AREA, region_id)
    
    IWW_IRR_1901_2014_all_ET = np.vstack((IWW_IRR_1901_2014_CESM2_ET_global,
                               IWW_IRR_1901_2014_CESM2_gw_ET_global,
                               IWW_IRR_1901_2014_NorESM_ET_global,
                               IWW_IRR_1901_2014_E3SM_ET_global,
                               IWW_IRR_1901_2014_IPSL_ET_global,
                                          IWW_IRR_1901_2014_MIROC_ET_global,
                               IWW_IRR_1901_2014_CNRM_ET_global))

    IWW_NOI_1901_2014_all_ET = np.vstack((IWW_NOI_1901_2014_CESM2_ET_global,
                                   IWW_NOI_1901_2014_CESM2_gw_ET_global,
                                   IWW_NOI_1901_2014_NorESM_ET_global,
                                   IWW_NOI_1901_2014_E3SM_ET_global,
                                   IWW_NOI_1901_2014_IPSL_ET_global,
                                          IWW_NOI_1901_2014_MIROC_ET_global,
                                   IWW_NOI_1901_2014_CNRM_ET_global))
    
    IWW_IRR_NOI_1901_2014_all_ET = IWW_IRR_1901_2014_all_ET - IWW_NOI_1901_2014_all_ET
    
    return IWW_IRR_1901_2014_all_ET, IWW_NOI_1901_2014_all_ET, IWW_IRR_NOI_1901_2014_all_ET

IWW_IRR_1901_2014_all_ET_SAS, IWW_NOI_1901_2014_all_ET_SAS, IWW_IRR_NOI_1901_2014_all_ET_SAS = get_regional_ET(38)
# IWW_IRR_1901_2014_all_ET_SAS_75 = np.percentile(IWW_IRR_1901_2014_all_ET_SAS, 100, axis = 0)
# IWW_IRR_1901_2014_all_ET_SAS_50 = np.percentile(IWW_IRR_1901_2014_all_ET_SAS, 50, axis = 0)
# IWW_IRR_1901_2014_all_ET_SAS_25 = np.percentile(IWW_IRR_1901_2014_all_ET_SAS, 0, axis = 0)

# IWW_NOI_1901_2014_all_ET_SAS_75 = np.percentile(IWW_NOI_1901_2014_all_ET_SAS, 100, axis = 0)
# IWW_NOI_1901_2014_all_ET_SAS_50 = np.percentile(IWW_NOI_1901_2014_all_ET_SAS, 50, axis = 0)
# IWW_NOI_1901_2014_all_ET_SAS_25 = np.percentile(IWW_NOI_1901_2014_all_ET_SAS, 0, axis = 0)

# IWW_IRR_NOI_1901_2014_all_ET_SAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_SAS, 100, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_SAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_SAS, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_SAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_SAS, 0, axis = 0)

IWW_IRR_1901_2014_all_ET_SAS_75 = np.percentile(IWW_IRR_1901_2014_all_ET_SAS, 75, axis = 0)
IWW_IRR_1901_2014_all_ET_SAS_50 = np.percentile(IWW_IRR_1901_2014_all_ET_SAS, 50, axis = 0)
IWW_IRR_1901_2014_all_ET_SAS_25 = np.percentile(IWW_IRR_1901_2014_all_ET_SAS, 25, axis = 0)
IWW_IRR_1901_2014_all_ET_SAS_mean = np.mean(IWW_IRR_1901_2014_all_ET_SAS, axis = 0)

IWW_NOI_1901_2014_all_ET_SAS_75 = np.percentile(IWW_NOI_1901_2014_all_ET_SAS, 75, axis = 0)
IWW_NOI_1901_2014_all_ET_SAS_50 = np.percentile(IWW_NOI_1901_2014_all_ET_SAS, 50, axis = 0)
IWW_NOI_1901_2014_all_ET_SAS_25 = np.percentile(IWW_NOI_1901_2014_all_ET_SAS, 25, axis = 0)
IWW_NOI_1901_2014_all_ET_SAS_mean = np.mean(IWW_NOI_1901_2014_all_ET_SAS, axis = 0)

IWW_IRR_NOI_1901_2014_all_ET_SAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_SAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_SAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_SAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_SAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_SAS, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_SAS_mean = np.mean(IWW_IRR_NOI_1901_2014_all_ET_SAS, axis = 0)

IWW_IRR_1901_2014_all_ET_EAS, IWW_NOI_1901_2014_all_ET_EAS, IWW_IRR_NOI_1901_2014_all_ET_EAS = get_regional_ET(36)
# IWW_IRR_1901_2014_all_ET_EAS_75 = np.percentile(IWW_IRR_1901_2014_all_ET_EAS, 100, axis = 0)
# IWW_IRR_1901_2014_all_ET_EAS_50 = np.percentile(IWW_IRR_1901_2014_all_ET_EAS, 50, axis = 0)
# IWW_IRR_1901_2014_all_ET_EAS_25 = np.percentile(IWW_IRR_1901_2014_all_ET_EAS, 0, axis = 0)

# IWW_NOI_1901_2014_all_ET_EAS_75 = np.percentile(IWW_NOI_1901_2014_all_ET_EAS, 100, axis = 0)
# IWW_NOI_1901_2014_all_ET_EAS_50 = np.percentile(IWW_NOI_1901_2014_all_ET_EAS, 50, axis = 0)
# IWW_NOI_1901_2014_all_ET_EAS_25 = np.percentile(IWW_NOI_1901_2014_all_ET_EAS, 0, axis = 0)

# IWW_IRR_NOI_1901_2014_all_ET_EAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_EAS, 100, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_EAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_EAS, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_EAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_EAS, 0, axis = 0)

IWW_IRR_1901_2014_all_ET_EAS_75 = np.percentile(IWW_IRR_1901_2014_all_ET_EAS, 75, axis = 0)
IWW_IRR_1901_2014_all_ET_EAS_50 = np.percentile(IWW_IRR_1901_2014_all_ET_EAS, 50, axis = 0)
IWW_IRR_1901_2014_all_ET_EAS_25 = np.percentile(IWW_IRR_1901_2014_all_ET_EAS, 25, axis = 0)


IWW_NOI_1901_2014_all_ET_EAS_75 = np.percentile(IWW_NOI_1901_2014_all_ET_EAS, 75, axis = 0)
IWW_NOI_1901_2014_all_ET_EAS_50 = np.percentile(IWW_NOI_1901_2014_all_ET_EAS, 50, axis = 0)
IWW_NOI_1901_2014_all_ET_EAS_25 = np.percentile(IWW_NOI_1901_2014_all_ET_EAS, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_ET_EAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_EAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_EAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_EAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_EAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_EAS, 25, axis = 0)

IWW_IRR_1901_2014_all_ET_MED, IWW_NOI_1901_2014_all_ET_MED, IWW_IRR_NOI_1901_2014_all_ET_MED = get_regional_ET(20)
# IWW_IRR_1901_2014_all_ET_MED_75 = np.percentile(IWW_IRR_1901_2014_all_ET_MED, 100, axis = 0)
# IWW_IRR_1901_2014_all_ET_MED_50 = np.percentile(IWW_IRR_1901_2014_all_ET_MED, 50, axis = 0)
# IWW_IRR_1901_2014_all_ET_MED_25 = np.percentile(IWW_IRR_1901_2014_all_ET_MED, 0, axis = 0)

# IWW_NOI_1901_2014_all_ET_MED_75 = np.percentile(IWW_NOI_1901_2014_all_ET_MED, 100, axis = 0)
# IWW_NOI_1901_2014_all_ET_MED_50 = np.percentile(IWW_NOI_1901_2014_all_ET_MED, 50, axis = 0)
# IWW_NOI_1901_2014_all_ET_MED_25 = np.percentile(IWW_NOI_1901_2014_all_ET_MED, 0, axis = 0)

# IWW_IRR_NOI_1901_2014_all_ET_MED_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_MED, 100, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_MED_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_MED, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_MED_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_MED, 0, axis = 0)

IWW_IRR_1901_2014_all_ET_MED_75 = np.percentile(IWW_IRR_1901_2014_all_ET_MED, 75, axis = 0)
IWW_IRR_1901_2014_all_ET_MED_50 = np.percentile(IWW_IRR_1901_2014_all_ET_MED, 50, axis = 0)
IWW_IRR_1901_2014_all_ET_MED_25 = np.percentile(IWW_IRR_1901_2014_all_ET_MED, 25, axis = 0)
IWW_IRR_1901_2014_all_ET_MED_mean = np.mean(IWW_IRR_1901_2014_all_ET_MED, axis = 0)

IWW_NOI_1901_2014_all_ET_MED_75 = np.percentile(IWW_NOI_1901_2014_all_ET_MED, 75, axis = 0)
IWW_NOI_1901_2014_all_ET_MED_50 = np.percentile(IWW_NOI_1901_2014_all_ET_MED, 50, axis = 0)
IWW_NOI_1901_2014_all_ET_MED_25 = np.percentile(IWW_NOI_1901_2014_all_ET_MED, 25, axis = 0)
IWW_NOI_1901_2014_all_ET_MED_mean = np.mean(IWW_NOI_1901_2014_all_ET_MED, axis = 0)

IWW_IRR_NOI_1901_2014_all_ET_MED_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_MED, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_MED_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_MED, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_MED_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_MED, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_MED_mean = np.mean(IWW_IRR_NOI_1901_2014_all_ET_MED, axis = 0)



IWW_IRR_1901_2014_all_ET_WNA, IWW_NOI_1901_2014_all_ET_WNA, IWW_IRR_NOI_1901_2014_all_ET_WNA = get_regional_ET(4)
# IWW_IRR_1901_2014_all_ET_WNA_75 = np.percentile(IWW_IRR_1901_2014_all_ET_WNA, 100, axis = 0)
# IWW_IRR_1901_2014_all_ET_WNA_50 = np.percentile(IWW_IRR_1901_2014_all_ET_WNA, 50, axis = 0)
# IWW_IRR_1901_2014_all_ET_WNA_25 = np.percentile(IWW_IRR_1901_2014_all_ET_WNA, 0, axis = 0)

# IWW_NOI_1901_2014_all_ET_WNA_75 = np.percentile(IWW_NOI_1901_2014_all_ET_WNA, 100, axis = 0)
# IWW_NOI_1901_2014_all_ET_WNA_50 = np.percentile(IWW_NOI_1901_2014_all_ET_WNA, 50, axis = 0)
# IWW_NOI_1901_2014_all_ET_WNA_25 = np.percentile(IWW_NOI_1901_2014_all_ET_WNA, 0, axis = 0)

# IWW_IRR_NOI_1901_2014_all_ET_WNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_WNA, 100, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_WNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_WNA, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_WNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_WNA, 0, axis = 0)

IWW_IRR_1901_2014_all_ET_WNA_75 = np.percentile(IWW_IRR_1901_2014_all_ET_WNA, 75, axis = 0)
IWW_IRR_1901_2014_all_ET_WNA_50 = np.percentile(IWW_IRR_1901_2014_all_ET_WNA, 50, axis = 0)
IWW_IRR_1901_2014_all_ET_WNA_25 = np.percentile(IWW_IRR_1901_2014_all_ET_WNA, 25, axis = 0)


IWW_NOI_1901_2014_all_ET_WNA_75 = np.percentile(IWW_NOI_1901_2014_all_ET_WNA, 75, axis = 0)
IWW_NOI_1901_2014_all_ET_WNA_50 = np.percentile(IWW_NOI_1901_2014_all_ET_WNA, 50, axis = 0)
IWW_NOI_1901_2014_all_ET_WNA_25 = np.percentile(IWW_NOI_1901_2014_all_ET_WNA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_ET_WNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_WNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_WNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_WNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_WNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_WNA, 25, axis = 0)


IWW_IRR_1901_2014_all_ET_CNA, IWW_NOI_1901_2014_all_ET_CNA, IWW_IRR_NOI_1901_2014_all_ET_CNA = get_regional_ET(5)
# IWW_IRR_1901_2014_all_ET_CNA_75 = np.percentile(IWW_IRR_1901_2014_all_ET_CNA, 100, axis = 0)
# IWW_IRR_1901_2014_all_ET_CNA_50 = np.percentile(IWW_IRR_1901_2014_all_ET_CNA, 50, axis = 0)
# IWW_IRR_1901_2014_all_ET_CNA_25 = np.percentile(IWW_IRR_1901_2014_all_ET_CNA, 0, axis = 0)

# IWW_NOI_1901_2014_all_ET_CNA_75 = np.percentile(IWW_NOI_1901_2014_all_ET_CNA, 100, axis = 0)
# IWW_NOI_1901_2014_all_ET_CNA_50 = np.percentile(IWW_NOI_1901_2014_all_ET_CNA, 50, axis = 0)
# IWW_NOI_1901_2014_all_ET_CNA_25 = np.percentile(IWW_NOI_1901_2014_all_ET_CNA, 0, axis = 0)

# IWW_IRR_NOI_1901_2014_all_ET_CNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_CNA, 100, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_CNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_CNA, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_ET_CNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_CNA, 0, axis = 0)

IWW_IRR_1901_2014_all_ET_CNA_75 = np.percentile(IWW_IRR_1901_2014_all_ET_CNA, 75, axis = 0)
IWW_IRR_1901_2014_all_ET_CNA_50 = np.percentile(IWW_IRR_1901_2014_all_ET_CNA, 50, axis = 0)
IWW_IRR_1901_2014_all_ET_CNA_25 = np.percentile(IWW_IRR_1901_2014_all_ET_CNA, 25, axis = 0)
IWW_IRR_1901_2014_all_ET_CNA_mean = np.mean(IWW_IRR_1901_2014_all_ET_CNA, axis = 0)

IWW_NOI_1901_2014_all_ET_CNA_75 = np.percentile(IWW_NOI_1901_2014_all_ET_CNA, 75, axis = 0)
IWW_NOI_1901_2014_all_ET_CNA_50 = np.percentile(IWW_NOI_1901_2014_all_ET_CNA, 50, axis = 0)
IWW_NOI_1901_2014_all_ET_CNA_25 = np.percentile(IWW_NOI_1901_2014_all_ET_CNA, 25, axis = 0)
IWW_NOI_1901_2014_all_ET_CNA_mean = np.mean(IWW_NOI_1901_2014_all_ET_CNA, axis = 0)

IWW_IRR_NOI_1901_2014_all_ET_CNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_CNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_CNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_CNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_CNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_CNA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_CNA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_ET_CNA, axis = 0)

IWW_IRR_1901_2014_all_ET_NCA, IWW_NOI_1901_2014_all_ET_NCA, IWW_IRR_NOI_1901_2014_all_ET_NCA = get_regional_ET(33)
IWW_IRR_1901_2014_all_ET_NCA_75 = np.percentile(IWW_IRR_1901_2014_all_ET_NCA, 75, axis = 0)
IWW_IRR_1901_2014_all_ET_NCA_50 = np.percentile(IWW_IRR_1901_2014_all_ET_NCA, 50, axis = 0)
IWW_IRR_1901_2014_all_ET_NCA_25 = np.percentile(IWW_IRR_1901_2014_all_ET_NCA, 25, axis = 0)
IWW_IRR_1901_2014_all_ET_NCA_mean = np.mean(IWW_IRR_1901_2014_all_ET_NCA, axis = 0)

IWW_NOI_1901_2014_all_ET_NCA_75 = np.percentile(IWW_NOI_1901_2014_all_ET_NCA, 75, axis = 0)
IWW_NOI_1901_2014_all_ET_NCA_50 = np.percentile(IWW_NOI_1901_2014_all_ET_NCA, 50, axis = 0)
IWW_NOI_1901_2014_all_ET_NCA_25 = np.percentile(IWW_NOI_1901_2014_all_ET_NCA, 25, axis = 0)
IWW_NOI_1901_2014_all_ET_NCA_mean = np.mean(IWW_NOI_1901_2014_all_ET_NCA, axis = 0)

IWW_IRR_NOI_1901_2014_all_ET_NCA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_NCA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_NCA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_NCA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_NCA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_NCA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_NCA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_ET_NCA, axis = 0)


IWW_IRR_1901_2014_all_ET_SEA, IWW_NOI_1901_2014_all_ET_SEA, IWW_IRR_NOI_1901_2014_all_ET_SEA = get_regional_ET(39)
IWW_IRR_1901_2014_all_ET_SEA_75 = np.percentile(IWW_IRR_1901_2014_all_ET_SEA, 75, axis = 0)
IWW_IRR_1901_2014_all_ET_SEA_50 = np.percentile(IWW_IRR_1901_2014_all_ET_SEA, 50, axis = 0)
IWW_IRR_1901_2014_all_ET_SEA_25 = np.percentile(IWW_IRR_1901_2014_all_ET_SEA, 25, axis = 0)

IWW_NOI_1901_2014_all_ET_SEA_75 = np.percentile(IWW_NOI_1901_2014_all_ET_SEA, 75, axis = 0)
IWW_NOI_1901_2014_all_ET_SEA_50 = np.percentile(IWW_NOI_1901_2014_all_ET_SEA, 50, axis = 0)
IWW_NOI_1901_2014_all_ET_SEA_25 = np.percentile(IWW_NOI_1901_2014_all_ET_SEA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_ET_SEA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_SEA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_SEA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_SEA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_ET_SEA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_ET_SEA, 25, axis = 0)

In [7]:
def get_regional_R(region_id):
    IWW_IRR_1901_2014_CESM2_R_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_R, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_R_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_R, AREA, region_id)

    IWW_IRR_1901_2014_CESM2_gw_R_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_gw_R, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_gw_R_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_gw_R, AREA, region_id)

    IWW_IRR_1901_2014_NorESM_R_global = calcu_Water_region(IWW_IRR_1901_2014_NorESM_R, AREA, region_id)
    IWW_NOI_1901_2014_NorESM_R_global = calcu_Water_region(IWW_NOI_1901_2014_NorESM_R, AREA, region_id)

    IWW_IRR_1901_2014_E3SM_R_global = calcu_Water_region(IWW_IRR_1901_2014_E3SM_R, AREA, region_id)
    IWW_NOI_1901_2014_E3SM_R_global = calcu_Water_region(IWW_NOI_1901_2014_E3SM_R, AREA, region_id)

    IWW_IRR_1901_2014_IPSL_R_global = calcu_Water_region(IWW_IRR_1901_2014_IPSL_R, AREA, region_id)
    IWW_NOI_1901_2014_IPSL_R_global = calcu_Water_region(IWW_NOI_1901_2014_IPSL_R, AREA, region_id)

    IWW_IRR_1901_2014_CNRM_R_global = calcu_Water_region(IWW_IRR_1901_2014_CNRM_R, AREA, region_id)
    IWW_NOI_1901_2014_CNRM_R_global = calcu_Water_region(IWW_NOI_1901_2014_CNRM_R, AREA, region_id)
    
    IWW_IRR_1901_2014_MIROC_R_global = calcu_Water_region(IWW_IRR_1901_2014_MIROC_R, AREA, region_id)
    IWW_NOI_1901_2014_MIROC_R_global = calcu_Water_region(IWW_NOI_1901_2014_MIROC_R, AREA, region_id)
    
    IWW_IRR_1901_2014_all_R = np.vstack((IWW_IRR_1901_2014_CESM2_R_global,
                               IWW_IRR_1901_2014_CESM2_gw_R_global,
                               IWW_IRR_1901_2014_NorESM_R_global,
                               IWW_IRR_1901_2014_E3SM_R_global,
                               IWW_IRR_1901_2014_IPSL_R_global,
                                         IWW_IRR_1901_2014_MIROC_R_global,
                               IWW_IRR_1901_2014_CNRM_R_global))

    IWW_NOI_1901_2014_all_R = np.vstack((IWW_NOI_1901_2014_CESM2_R_global,
                                   IWW_NOI_1901_2014_CESM2_gw_R_global,
                                   IWW_NOI_1901_2014_NorESM_R_global,
                                   IWW_NOI_1901_2014_E3SM_R_global,
                                   IWW_NOI_1901_2014_IPSL_R_global,
                                         IWW_NOI_1901_2014_MIROC_R_global,
                                   IWW_NOI_1901_2014_CNRM_R_global))
    
    IWW_IRR_NOI_1901_2014_all_R = IWW_IRR_1901_2014_all_R - IWW_NOI_1901_2014_all_R
    
    return IWW_IRR_1901_2014_all_R, IWW_NOI_1901_2014_all_R, IWW_IRR_NOI_1901_2014_all_R

IWW_IRR_1901_2014_all_R_SAS, IWW_NOI_1901_2014_all_R_SAS, IWW_IRR_NOI_1901_2014_all_R_SAS = get_regional_R(38)


IWW_IRR_1901_2014_all_R_SAS_75 = np.percentile(IWW_IRR_1901_2014_all_R_SAS, 75, axis = 0)
IWW_IRR_1901_2014_all_R_SAS_50 = np.percentile(IWW_IRR_1901_2014_all_R_SAS, 50, axis = 0)
IWW_IRR_1901_2014_all_R_SAS_25 = np.percentile(IWW_IRR_1901_2014_all_R_SAS, 25, axis = 0)
IWW_IRR_1901_2014_all_R_SAS_mean = np.mean(IWW_IRR_1901_2014_all_R_SAS, axis = 0)

IWW_NOI_1901_2014_all_R_SAS_75 = np.percentile(IWW_NOI_1901_2014_all_R_SAS, 75, axis = 0)
IWW_NOI_1901_2014_all_R_SAS_50 = np.percentile(IWW_NOI_1901_2014_all_R_SAS, 50, axis = 0)
IWW_NOI_1901_2014_all_R_SAS_25 = np.percentile(IWW_NOI_1901_2014_all_R_SAS, 25, axis = 0)
IWW_NOI_1901_2014_all_R_SAS_mean = np.mean(IWW_NOI_1901_2014_all_R_SAS, axis = 0)

IWW_IRR_NOI_1901_2014_all_R_SAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_SAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_SAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_SAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_SAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_SAS, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_SAS_mean = np.mean(IWW_IRR_NOI_1901_2014_all_R_SAS, axis = 0)

IWW_IRR_1901_2014_all_R_EAS, IWW_NOI_1901_2014_all_R_EAS, IWW_IRR_NOI_1901_2014_all_R_EAS = get_regional_R(36)
IWW_IRR_1901_2014_all_R_EAS_75 = np.percentile(IWW_IRR_1901_2014_all_R_EAS, 75, axis = 0)
IWW_IRR_1901_2014_all_R_EAS_50 = np.percentile(IWW_IRR_1901_2014_all_R_EAS, 50, axis = 0)
IWW_IRR_1901_2014_all_R_EAS_25 = np.percentile(IWW_IRR_1901_2014_all_R_EAS, 25, axis = 0)
IWW_IRR_1901_2014_all_R_EAS_mean = np.mean(IWW_IRR_1901_2014_all_R_EAS, axis = 0)

IWW_NOI_1901_2014_all_R_EAS_75 = np.percentile(IWW_NOI_1901_2014_all_R_EAS, 75, axis = 0)
IWW_NOI_1901_2014_all_R_EAS_50 = np.percentile(IWW_NOI_1901_2014_all_R_EAS, 50, axis = 0)
IWW_NOI_1901_2014_all_R_EAS_25 = np.percentile(IWW_NOI_1901_2014_all_R_EAS, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_R_EAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_EAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_EAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_EAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_EAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_EAS, 25, axis = 0)

IWW_IRR_1901_2014_all_R_MED, IWW_NOI_1901_2014_all_R_MED, IWW_IRR_NOI_1901_2014_all_R_MED = get_regional_R(20)


IWW_IRR_1901_2014_all_R_MED_75 = np.percentile(IWW_IRR_1901_2014_all_R_MED, 75, axis = 0)
IWW_IRR_1901_2014_all_R_MED_50 = np.percentile(IWW_IRR_1901_2014_all_R_MED, 50, axis = 0)
IWW_IRR_1901_2014_all_R_MED_25 = np.percentile(IWW_IRR_1901_2014_all_R_MED, 25, axis = 0)
IWW_IRR_1901_2014_all_R_MED_mean = np.mean(IWW_IRR_1901_2014_all_R_MED, axis = 0)

IWW_NOI_1901_2014_all_R_MED_75 = np.percentile(IWW_NOI_1901_2014_all_R_MED, 75, axis = 0)
IWW_NOI_1901_2014_all_R_MED_50 = np.percentile(IWW_NOI_1901_2014_all_R_MED, 50, axis = 0)
IWW_NOI_1901_2014_all_R_MED_25 = np.percentile(IWW_NOI_1901_2014_all_R_MED, 25, axis = 0)
IWW_NOI_1901_2014_all_R_MED_mean = np.mean(IWW_NOI_1901_2014_all_R_MED, axis = 0)

IWW_IRR_NOI_1901_2014_all_R_MED_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_MED, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_MED_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_MED, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_MED_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_MED, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_MED_mean = np.mean(IWW_IRR_NOI_1901_2014_all_R_MED, axis = 0)



IWW_IRR_1901_2014_all_R_WNA, IWW_NOI_1901_2014_all_R_WNA, IWW_IRR_NOI_1901_2014_all_R_WNA = get_regional_R(4)


IWW_IRR_1901_2014_all_R_WNA_75 = np.percentile(IWW_IRR_1901_2014_all_R_WNA, 75, axis = 0)
IWW_IRR_1901_2014_all_R_WNA_50 = np.percentile(IWW_IRR_1901_2014_all_R_WNA, 50, axis = 0)
IWW_IRR_1901_2014_all_R_WNA_25 = np.percentile(IWW_IRR_1901_2014_all_R_WNA, 25, axis = 0)

IWW_NOI_1901_2014_all_R_WNA_75 = np.percentile(IWW_NOI_1901_2014_all_R_WNA, 75, axis = 0)
IWW_NOI_1901_2014_all_R_WNA_50 = np.percentile(IWW_NOI_1901_2014_all_R_WNA, 50, axis = 0)
IWW_NOI_1901_2014_all_R_WNA_25 = np.percentile(IWW_NOI_1901_2014_all_R_WNA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_R_WNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_WNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_WNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_WNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_WNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_WNA, 25, axis = 0)


IWW_IRR_1901_2014_all_R_CNA, IWW_NOI_1901_2014_all_R_CNA, IWW_IRR_NOI_1901_2014_all_R_CNA = get_regional_R(5)


IWW_IRR_1901_2014_all_R_CNA_75 = np.percentile(IWW_IRR_1901_2014_all_R_CNA, 75, axis = 0)
IWW_IRR_1901_2014_all_R_CNA_50 = np.percentile(IWW_IRR_1901_2014_all_R_CNA, 50, axis = 0)
IWW_IRR_1901_2014_all_R_CNA_25 = np.percentile(IWW_IRR_1901_2014_all_R_CNA, 25, axis = 0)
IWW_IRR_1901_2014_all_R_CNA_mean = np.mean(IWW_IRR_1901_2014_all_R_CNA, axis = 0)

IWW_NOI_1901_2014_all_R_CNA_75 = np.percentile(IWW_NOI_1901_2014_all_R_CNA, 75, axis = 0)
IWW_NOI_1901_2014_all_R_CNA_50 = np.percentile(IWW_NOI_1901_2014_all_R_CNA, 50, axis = 0)
IWW_NOI_1901_2014_all_R_CNA_25 = np.percentile(IWW_NOI_1901_2014_all_R_CNA, 25, axis = 0)
IWW_NOI_1901_2014_all_R_CNA_mean = np.mean(IWW_NOI_1901_2014_all_R_CNA, axis = 0)

IWW_IRR_NOI_1901_2014_all_R_CNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_CNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_CNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_CNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_CNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_CNA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_CNA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_R_CNA, axis = 0)


IWW_IRR_1901_2014_all_R_NCA, IWW_NOI_1901_2014_all_R_NCA, IWW_IRR_NOI_1901_2014_all_R_NCA = get_regional_R(33)
IWW_IRR_1901_2014_all_R_NCA_75 = np.percentile(IWW_IRR_1901_2014_all_R_NCA, 75, axis = 0)
IWW_IRR_1901_2014_all_R_NCA_50 = np.percentile(IWW_IRR_1901_2014_all_R_NCA, 50, axis = 0)
IWW_IRR_1901_2014_all_R_NCA_25 = np.percentile(IWW_IRR_1901_2014_all_R_NCA, 25, axis = 0)
IWW_IRR_1901_2014_all_R_NCA_mean = np.mean(IWW_IRR_1901_2014_all_R_NCA, axis = 0)

IWW_NOI_1901_2014_all_R_NCA_75 = np.percentile(IWW_NOI_1901_2014_all_R_NCA, 75, axis = 0)
IWW_NOI_1901_2014_all_R_NCA_50 = np.percentile(IWW_NOI_1901_2014_all_R_NCA, 50, axis = 0)
IWW_NOI_1901_2014_all_R_NCA_25 = np.percentile(IWW_NOI_1901_2014_all_R_NCA, 25, axis = 0)
IWW_NOI_1901_2014_all_R_NCA_mean = np.mean(IWW_NOI_1901_2014_all_R_NCA, axis = 0)

IWW_IRR_NOI_1901_2014_all_R_NCA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_NCA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_NCA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_NCA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_NCA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_NCA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_NCA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_R_NCA, axis = 0)


IWW_IRR_1901_2014_all_R_SEA, IWW_NOI_1901_2014_all_R_SEA, IWW_IRR_NOI_1901_2014_all_R_SEA = get_regional_R(39)
IWW_IRR_1901_2014_all_R_SEA_75 = np.percentile(IWW_IRR_1901_2014_all_R_SEA, 75, axis = 0)
IWW_IRR_1901_2014_all_R_SEA_50 = np.percentile(IWW_IRR_1901_2014_all_R_SEA, 50, axis = 0)
IWW_IRR_1901_2014_all_R_SEA_25 = np.percentile(IWW_IRR_1901_2014_all_R_SEA, 25, axis = 0)

IWW_NOI_1901_2014_all_R_SEA_75 = np.percentile(IWW_NOI_1901_2014_all_R_SEA, 75, axis = 0)
IWW_NOI_1901_2014_all_R_SEA_50 = np.percentile(IWW_NOI_1901_2014_all_R_SEA, 50, axis = 0)
IWW_NOI_1901_2014_all_R_SEA_25 = np.percentile(IWW_NOI_1901_2014_all_R_SEA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_R_SEA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_SEA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_SEA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_SEA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_R_SEA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_R_SEA, 25, axis = 0)

In [8]:
def get_19_years_mean(data):
    data_1=np.mean(data[0:19])
    data_2=np.mean(data[19:38])
    data_3=np.mean(data[38:57])
    data_4=np.mean(data[57:76])
    data_5=np.mean(data[76:95])
    data_6=np.mean(data[95:114])
    return np.array([data_1,data_2,data_3,data_4,data_5,data_6])

In [9]:
IRR_P_SAS_50 = get_19_years_mean(IWW_IRR_1901_2014_all_P_SAS_50)
IRR_P_SAS_mean = get_19_years_mean(IWW_IRR_1901_2014_all_P_SAS_mean)
IRR_P_SAS_75 = get_19_years_mean(IWW_IRR_1901_2014_all_P_SAS_75)
IRR_P_SAS_25 = get_19_years_mean(IWW_IRR_1901_2014_all_P_SAS_25)

IRR_P_MED_50 = get_19_years_mean(IWW_IRR_1901_2014_all_P_MED_50)
IRR_P_MED_mean = get_19_years_mean(IWW_IRR_1901_2014_all_P_MED_mean)
IRR_P_MED_75 = get_19_years_mean(IWW_IRR_1901_2014_all_P_MED_75)
IRR_P_MED_25 = get_19_years_mean(IWW_IRR_1901_2014_all_P_MED_25)

IRR_P_CNA_50 = get_19_years_mean(IWW_IRR_1901_2014_all_P_CNA_50)
IRR_P_CNA_mean = get_19_years_mean(IWW_IRR_1901_2014_all_P_CNA_mean)
IRR_P_CNA_75 = get_19_years_mean(IWW_IRR_1901_2014_all_P_CNA_75)
IRR_P_CNA_25 = get_19_years_mean(IWW_IRR_1901_2014_all_P_CNA_25)

IRR_P_NCA_50 = get_19_years_mean(IWW_IRR_1901_2014_all_P_NCA_50)
IRR_P_NCA_mean = get_19_years_mean(IWW_IRR_1901_2014_all_P_NCA_mean)
IRR_P_NCA_75 = get_19_years_mean(IWW_IRR_1901_2014_all_P_NCA_75)
IRR_P_NCA_25 = get_19_years_mean(IWW_IRR_1901_2014_all_P_NCA_25)


IRR_ET_SAS_50 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_SAS_50)
IRR_ET_SAS_mean = get_19_years_mean(IWW_IRR_1901_2014_all_ET_SAS_mean)
IRR_ET_SAS_75 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_SAS_75)
IRR_ET_SAS_25 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_SAS_25)

IRR_ET_MED_50 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_MED_50)
IRR_ET_MED_mean = get_19_years_mean(IWW_IRR_1901_2014_all_ET_MED_mean)
IRR_ET_MED_75 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_MED_75)
IRR_ET_MED_25 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_MED_25)

IRR_ET_CNA_50 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_CNA_50)
IRR_ET_CNA_mean = get_19_years_mean(IWW_IRR_1901_2014_all_ET_CNA_mean)
IRR_ET_CNA_75 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_CNA_75)
IRR_ET_CNA_25 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_CNA_25)

IRR_ET_NCA_50 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_NCA_50)
IRR_ET_NCA_mean = get_19_years_mean(IWW_IRR_1901_2014_all_ET_NCA_mean)
IRR_ET_NCA_75 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_NCA_75)
IRR_ET_NCA_25 = get_19_years_mean(IWW_IRR_1901_2014_all_ET_NCA_25)



IRR_R_SAS_50 = get_19_years_mean(IWW_IRR_1901_2014_all_R_SAS_50)
IRR_R_SAS_mean = get_19_years_mean(IWW_IRR_1901_2014_all_R_SAS_mean)
IRR_R_SAS_75 = get_19_years_mean(IWW_IRR_1901_2014_all_R_SAS_75)
IRR_R_SAS_25 = get_19_years_mean(IWW_IRR_1901_2014_all_R_SAS_25)

IRR_R_MED_50 = get_19_years_mean(IWW_IRR_1901_2014_all_R_MED_50)
IRR_R_MED_mean = get_19_years_mean(IWW_IRR_1901_2014_all_R_MED_mean)
IRR_R_MED_75 = get_19_years_mean(IWW_IRR_1901_2014_all_R_MED_75)
IRR_R_MED_25 = get_19_years_mean(IWW_IRR_1901_2014_all_R_MED_25)

IRR_R_CNA_50 = get_19_years_mean(IWW_IRR_1901_2014_all_R_CNA_50)
IRR_R_CNA_mean = get_19_years_mean(IWW_IRR_1901_2014_all_R_CNA_mean)
IRR_R_CNA_75 = get_19_years_mean(IWW_IRR_1901_2014_all_R_CNA_75)
IRR_R_CNA_25 = get_19_years_mean(IWW_IRR_1901_2014_all_R_CNA_25)

IRR_R_NCA_50 = get_19_years_mean(IWW_IRR_1901_2014_all_R_NCA_50)
IRR_R_NCA_mean = get_19_years_mean(IWW_IRR_1901_2014_all_R_NCA_mean)
IRR_R_NCA_75 = get_19_years_mean(IWW_IRR_1901_2014_all_R_NCA_75)
IRR_R_NCA_25 = get_19_years_mean(IWW_IRR_1901_2014_all_R_NCA_25)

In [10]:
NOI_P_SAS_50 = get_19_years_mean(IWW_NOI_1901_2014_all_P_SAS_50)
NOI_P_SAS_mean = get_19_years_mean(IWW_NOI_1901_2014_all_P_SAS_mean)
NOI_P_SAS_75 = get_19_years_mean(IWW_NOI_1901_2014_all_P_SAS_75)
NOI_P_SAS_25 = get_19_years_mean(IWW_NOI_1901_2014_all_P_SAS_25)

NOI_P_MED_50 = get_19_years_mean(IWW_NOI_1901_2014_all_P_MED_50)
NOI_P_MED_mean = get_19_years_mean(IWW_NOI_1901_2014_all_P_MED_mean)
NOI_P_MED_75 = get_19_years_mean(IWW_NOI_1901_2014_all_P_MED_75)
NOI_P_MED_25 = get_19_years_mean(IWW_NOI_1901_2014_all_P_MED_25)

NOI_P_CNA_50 = get_19_years_mean(IWW_NOI_1901_2014_all_P_CNA_50)
NOI_P_CNA_mean = get_19_years_mean(IWW_NOI_1901_2014_all_P_CNA_mean)
NOI_P_CNA_75 = get_19_years_mean(IWW_NOI_1901_2014_all_P_CNA_75)
NOI_P_CNA_25 = get_19_years_mean(IWW_NOI_1901_2014_all_P_CNA_25)

NOI_P_NCA_50 = get_19_years_mean(IWW_NOI_1901_2014_all_P_NCA_50)
NOI_P_NCA_mean = get_19_years_mean(IWW_NOI_1901_2014_all_P_NCA_mean)
NOI_P_NCA_75 = get_19_years_mean(IWW_NOI_1901_2014_all_P_NCA_75)
NOI_P_NCA_25 = get_19_years_mean(IWW_NOI_1901_2014_all_P_NCA_25)


NOI_ET_SAS_50 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_SAS_50)
NOI_ET_SAS_mean = get_19_years_mean(IWW_NOI_1901_2014_all_ET_SAS_mean)
NOI_ET_SAS_75 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_SAS_75)
NOI_ET_SAS_25 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_SAS_25)

NOI_ET_MED_50 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_MED_50)
NOI_ET_MED_mean = get_19_years_mean(IWW_NOI_1901_2014_all_ET_MED_mean)
NOI_ET_MED_75 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_MED_75)
NOI_ET_MED_25 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_MED_25)

NOI_ET_CNA_50 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_CNA_50)
NOI_ET_CNA_mean = get_19_years_mean(IWW_NOI_1901_2014_all_ET_CNA_mean)
NOI_ET_CNA_75 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_CNA_75)
NOI_ET_CNA_25 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_CNA_25)

NOI_ET_NCA_50 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_NCA_50)
NOI_ET_NCA_mean = get_19_years_mean(IWW_NOI_1901_2014_all_ET_NCA_mean)
NOI_ET_NCA_75 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_NCA_75)
NOI_ET_NCA_25 = get_19_years_mean(IWW_NOI_1901_2014_all_ET_NCA_25)



NOI_R_SAS_50 = get_19_years_mean(IWW_NOI_1901_2014_all_R_SAS_50)
NOI_R_SAS_mean = get_19_years_mean(IWW_NOI_1901_2014_all_R_SAS_mean)
NOI_R_SAS_75 = get_19_years_mean(IWW_NOI_1901_2014_all_R_SAS_75)
NOI_R_SAS_25 = get_19_years_mean(IWW_NOI_1901_2014_all_R_SAS_25)

NOI_R_MED_50 = get_19_years_mean(IWW_NOI_1901_2014_all_R_MED_50)
NOI_R_MED_mean = get_19_years_mean(IWW_NOI_1901_2014_all_R_MED_mean)
NOI_R_MED_75 = get_19_years_mean(IWW_NOI_1901_2014_all_R_MED_75)
NOI_R_MED_25 = get_19_years_mean(IWW_NOI_1901_2014_all_R_MED_25)

NOI_R_CNA_50 = get_19_years_mean(IWW_NOI_1901_2014_all_R_CNA_50)
NOI_R_CNA_mean = get_19_years_mean(IWW_NOI_1901_2014_all_R_CNA_mean)
NOI_R_CNA_75 = get_19_years_mean(IWW_NOI_1901_2014_all_R_CNA_75)
NOI_R_CNA_25 = get_19_years_mean(IWW_NOI_1901_2014_all_R_CNA_25)

NOI_R_NCA_50 = get_19_years_mean(IWW_NOI_1901_2014_all_R_NCA_50)
NOI_R_NCA_mean = get_19_years_mean(IWW_NOI_1901_2014_all_R_NCA_mean)
NOI_R_NCA_75 = get_19_years_mean(IWW_NOI_1901_2014_all_R_NCA_75)
NOI_R_NCA_25 = get_19_years_mean(IWW_NOI_1901_2014_all_R_NCA_25)

In [ ]:
fig = plt.figure(figsize=(12, 20),dpi=300)
fig.subplots_adjust(hspace=0.3, wspace=0.3, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)
ax1 = plt.subplot(411, frameon=True)
# ax1.spines['top'].set_visible(False)
# ax1.spines['right'].set_visible(False)
X = np.array([1910, 1929, 1948, 1967, 1986, 2005])
plt.bar(X-6, IRR_P_SAS_mean, yerr=[IRR_P_SAS_mean-IRR_P_SAS_25, IRR_P_SAS_75-IRR_P_SAS_mean], width = 2, color='dodgerblue', capsize=6,label='P tranirr',alpha=0.8)
plt.bar(X-4, NOI_P_SAS_mean, yerr=[NOI_P_SAS_mean-NOI_P_SAS_25, NOI_P_SAS_75-NOI_P_SAS_mean], width = 2, color='aqua', capsize=6,label='P 1901irr',alpha=0.8)

plt.bar(X-1, IRR_ET_SAS_mean, yerr=[IRR_ET_SAS_mean-IRR_ET_SAS_25, IRR_ET_SAS_75-IRR_ET_SAS_mean], width = 2, color='tomato', capsize=6,label='ET tranirr',alpha=0.8)
plt.bar(X+1, NOI_ET_SAS_mean, yerr=[NOI_ET_SAS_mean-NOI_ET_SAS_25, NOI_ET_SAS_75-NOI_ET_SAS_mean], width = 2, color='darksalmon', capsize=6,label='ET 1901irr',alpha=0.8)

plt.bar(X+4, IRR_R_SAS_mean, yerr=[IRR_R_SAS_mean-IRR_R_SAS_25, IRR_R_SAS_75-IRR_R_SAS_mean], width = 2, color='lime', capsize=6,label='R tranirr',alpha=0.8)
plt.bar(X+6, NOI_R_SAS_mean, yerr=[NOI_R_SAS_mean-NOI_R_SAS_25, NOI_R_SAS_75-NOI_R_SAS_mean], width = 2, color='lightgreen', capsize=6,label='R 1901irr',alpha=0.8)
plt.xticks(X, ['1901-1919','1920-1938','1939-1957','1958-1976','1977-1995','1996-2014'], fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=12)
plt.xlim(1901, 2025)
plt.ylabel('Water fluxes (mm/year)', fontsize=16)
plt.title('South Asia', fontsize=20, loc='center')
plt.title('a', fontsize=14, loc='left')


ax1 = plt.subplot(412, frameon=True)
plt.bar(X-6, IRR_P_MED_mean, yerr=[IRR_P_MED_mean-IRR_P_MED_25, IRR_P_MED_75-IRR_P_MED_mean], width = 2, color='dodgerblue', capsize=6,label='P tranirr',alpha=0.8)
plt.bar(X-4, NOI_P_MED_mean, yerr=[NOI_P_MED_mean-NOI_P_MED_25, NOI_P_MED_75-NOI_P_MED_mean], width = 2, color='aqua', capsize=6,label='P 1901irr',alpha=0.8)

plt.bar(X-1, IRR_ET_MED_mean, yerr=[IRR_ET_MED_mean-IRR_ET_MED_25, IRR_ET_MED_75-IRR_ET_MED_mean], width = 2, color='tomato', capsize=6,label='ET tranirr',alpha=0.8)
plt.bar(X+1, NOI_ET_MED_mean, yerr=[NOI_ET_MED_mean-NOI_ET_MED_25, NOI_ET_MED_75-NOI_ET_MED_mean], width = 2, color='darksalmon', capsize=6,label='ET 1901irr',alpha=0.8)

plt.bar(X+4, IRR_R_MED_mean, yerr=[IRR_R_MED_mean-IRR_R_MED_25, IRR_R_MED_75-IRR_R_MED_mean], width = 2, color='lime', capsize=6,label='R tranirr',alpha=0.8)
plt.bar(X+6, NOI_R_MED_mean, yerr=[NOI_R_MED_mean-NOI_R_MED_25, NOI_R_MED_75-NOI_R_MED_mean], width = 2, color='lightgreen', capsize=6,label='R 1901irr',alpha=0.8)
plt.xticks(X, ['1901-1919','1920-1938','1939-1957','1958-1976','1977-1995','1996-2014'], fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=12)
plt.xlim(1901, 2025)
plt.ylabel('Water fluxes (mm/year)', fontsize=16)
plt.title('Mediterranean', fontsize=20, loc='center')
plt.title('b', fontsize=14, loc='left')

ax1 = plt.subplot(413, frameon=True)
plt.bar(X-6, IRR_P_CNA_mean, yerr=[IRR_P_CNA_mean-IRR_P_CNA_25, IRR_P_CNA_75-IRR_P_CNA_mean], width = 2, color='dodgerblue', capsize=6,label='P tranirr',alpha=0.8)
plt.bar(X-4, NOI_P_CNA_mean, yerr=[NOI_P_CNA_mean-NOI_P_CNA_25, NOI_P_CNA_75-NOI_P_CNA_mean], width = 2, color='aqua', capsize=6,label='P 1901irr',alpha=0.8)

plt.bar(X-1, IRR_ET_CNA_mean, yerr=[IRR_ET_CNA_mean-IRR_ET_CNA_25, IRR_ET_CNA_75-IRR_ET_CNA_mean], width = 2, color='tomato', capsize=6,label='ET tranirr',alpha=0.8)
plt.bar(X+1, NOI_ET_CNA_mean, yerr=[NOI_ET_CNA_mean-NOI_ET_CNA_25, NOI_ET_CNA_75-NOI_ET_CNA_mean], width = 2, color='darksalmon', capsize=6,label='ET 1901irr',alpha=0.8)

plt.bar(X+4, IRR_R_CNA_mean, yerr=[IRR_R_CNA_mean-IRR_R_CNA_25, IRR_R_CNA_75-IRR_R_CNA_mean], width = 2, color='lime', capsize=6,label='R tranirr',alpha=0.8)
plt.bar(X+6, NOI_R_CNA_mean, yerr=[NOI_R_CNA_mean-NOI_R_CNA_25, NOI_R_CNA_75-NOI_R_CNA_mean], width = 2, color='lightgreen', capsize=6,label='R 1901irr',alpha=0.8)
plt.xticks(X, ['1901-1919','1920-1938','1939-1957','1958-1976','1977-1995','1996-2014'], fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=12)
plt.xlim(1901, 2025)
plt.ylabel('Water fluxes (mm/year)', fontsize=16)
plt.title('Central North America', fontsize=20, loc='center')
plt.title('c', fontsize=14, loc='left')

ax1 = plt.subplot(414, frameon=True)
plt.bar(X-6, IRR_P_NCA_mean, yerr=[IRR_P_NCA_mean-IRR_P_NCA_25, IRR_P_NCA_75-IRR_P_NCA_mean], width = 2, color='dodgerblue', capsize=6,label='P tranirr',alpha=0.8)
plt.bar(X-4, NOI_P_NCA_mean, yerr=[NOI_P_NCA_mean-NOI_P_NCA_25, NOI_P_NCA_75-NOI_P_NCA_mean], width = 2, color='aqua', capsize=6,label='P 1901irr',alpha=0.8)

plt.bar(X-1, IRR_ET_NCA_mean, yerr=[IRR_ET_NCA_mean-IRR_ET_NCA_25, IRR_ET_NCA_75-IRR_ET_NCA_mean], width = 2, color='tomato', capsize=6,label='ET tranirr',alpha=0.8)
plt.bar(X+1, NOI_ET_NCA_mean, yerr=[NOI_ET_NCA_mean-NOI_ET_NCA_25, NOI_ET_NCA_75-NOI_ET_NCA_mean], width = 2, color='darksalmon', capsize=6,label='ET 1901irr',alpha=0.8)

plt.bar(X+4, IRR_R_NCA_mean, yerr=[IRR_R_NCA_mean-IRR_R_NCA_25, IRR_R_NCA_75-IRR_R_NCA_mean], width = 2, color='lime', capsize=6,label='R tranirr',alpha=0.8)
plt.bar(X+6, NOI_R_NCA_mean, yerr=[NOI_R_NCA_mean-NOI_R_NCA_25, NOI_R_NCA_75-NOI_R_NCA_mean], width = 2, color='lightgreen', capsize=6,label='R 1901irr',alpha=0.8)
plt.xticks(X, ['1901-1919','1920-1938','1939-1957','1958-1976','1977-1995','1996-2014'], fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=12)
plt.xlim(1901, 2025)
plt.ylabel('Water fluxes (mm/year)', fontsize=16)
plt.title('West Central Asia', fontsize=20, loc='center')
plt.title('d', fontsize=14, loc='left')